In [1]:
import pprint
import json
import datetime

import h5py

import numpy as np
np.random.seed(17) # fix NumPy random number generator seed

import networkx as nx
import matplotlib.pyplot as plt


# add some relative paths so we can import sampling.py module.
import sys
sys.path.append("../")
sys.path.append("../src")
sys_path = sys.path

import sampling

# Benchmark Workflow

The *benchmark performer* shall:
1. Select an `instance.<uuid-x>.json` file from the `benchmark_instances` directory.
2. Generate samples from the instance using their method and hardware of choice.  The `instance.<uuid-x>.json` file includes the appropriate time limit and number of samples $k$ to generate.
3. Write the samples they generated to `solution.<uuid-y>.hdf5` file in the `benchmark_performer_submissions` folder.  Metadata fields inside of the `solution.<uuid-y>.hdf5` reference the original `instance.<uuid-x>.json` file and UUID.


The *benchmark proctor* shall:
1. Select a `solution.<uuid-y>.hdf5` file and perform verification that the $k$ samples were accurately drawn from the system of interest.
2. Write out performance metrics to a `performance_metrics.<uuid-z>.json` file in the `benchmark_performance_metrics` directory.


Optionally:  a script will parse `performance_metrics.*.json` files in the `benchmark_performance_metrics` directory and present summary statistics, plots, etc.


In [2]:
# Acting as the Benchmark Proctor:
#
# Fetch the solution file submitted by the benchmark performer


solution_uuid = "ee69de1e-3620-11ee-be05-8f4ea6fc831a" 
solution_filename = "../benchmark_performer_submissions/solution." + solution_uuid + ".h5"

with h5py.File(solution_filename, "r") as f:
    print("Benchmark Instance UUID:",f["solution"]["instance_uuid"][()])
    print("Benchmark Performer Solution UUID:",f["solution"]["solution_uuid"][()])

    states_observed = f["solution"]["solution_reported"]["states_observed"][()]
    states_observed_counts = f["solution"]["solution_reported"]["states_observed_counts"][()]

    f.close()

    


Benchmark Instance UUID: b'a8be0d9a-3238-11ee-b3fb-9309595bcf35'
Benchmark Performer Solution UUID: b'ee69de1e-3620-11ee-be05-8f4ea6fc831a'


In [3]:


k_returned = states_observed_counts.sum()

# TODO: calculate average time per sample.  including number of CPUs used if parallelized.  

print("Num samples returned from benchmark performer:",k_returned)

#### TODO:  implement verification methods from https://github.com/lanl-ansi/GraphicalModelLearning.jl

Num samples returned from benchmark performer: 1000.0
